In [13]:
from itertools import combinations
import math

In [9]:
data = {
    0: [(0,0),0],
    1: [(-3,3),5],
    2: [(1,11),4],
    3: [(4,7),3],
    4: [(-5,9),6],
    5: [(-5,-2),7],
    6: [(-4,-7),3],
    7: [(6,0),4],
    8: [(3,-6),6],
    9: [(-1,-3),5],
    10: [(0,-6),4],
    11: [(6,4),7],
    12: [(2,5),3],
    13: [(-2,8),4],
    14: [(6,10),5],
    15: [(1,8),6],
    16: [(-3,1),8],
    17: [(-6,5),5],
    18: [(2,9),7],
    19: [(-6,-5),6],
    20: [(5,-4),6]
}

In [10]:
# List of farm  including the depot
farms = [*range(0,21)]

# List of farms that are visited everyday
everyDay = [*range(0,10)]

# List of farms that are visited each other day
otherDay = [*range(10,21)]

# List of day types
dayType = [1,2]

# Tanker lorry capacity (1,000)
tankerCap = 80

# Every day farms requirements
everyDayReq = 0
for i in everyDay:
    everyDayReq += data[i][1]

In [14]:
# Compute pairwise distance matrix
# numpy linalg norm = euclidean n=2

def distance(city1, city2):
    c1 = data[city1][0]
    c2 = data[city2][0]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(farms, 2)}

In [16]:
import pyscipopt as scip

In [65]:
m = scip.Model()
edges = list(dist.keys())

Decision variables 

In [66]:
# Edge variables = 1, if farm 'i' is adjacent to farm 'j' on the tour of day type 'k'.
x = dict()
for i,j in edges:
    for k in dayType:
        x[(i,j,k)] = m.addVar(vtype="B",name="x(%s,%s,%s)"%(i,j,k))
# Other day variables = 1, if farm 'i' is visited on the tour of day type 'k'.
y = dict()
for i in otherDay:
    for k in dayType:
        y[(i,k)] = m.addVar(vtype="B",name="y(%s,%s)"%(i,k))
# Symmetry constraints: copy the object (not a constraint)
for i,j,k in list(x.keys()):
    x[(j, i, k)] = x[(i, j, k)]


constraints

In [67]:
# 1. Every day constraints: two edges incident to an every day farm on tour of day type 'k'. 
for i in otherDay:
    for k in dayType:
        m.addCons(scip.quicksum(x[(i,j,k)] for j in farms if (i,j) in edges) == 2)

In [68]:
# Other day constraints: two edges incident to an other day farm on tour of day type 'k'.
for i in otherDay:
    for k in dayType:
        m.addCons(scip.quicksum(x[(i,j,k)] for j in farms if (i,j) in edges) == 2*y[(i,k)])

In [69]:
# Tanker capacity constraint.
for k in dayType:
    m.addCons(scip.quicksum(data[i][1]*y[(i,k)] for i in otherDay) <= tankerCap - everyDayReq)

In [70]:
# Other day farms are visited on day type 1 or 2.
for i in otherDay:
    m.addCons(scip.quicksum(y[(i,k)] for k in dayType) == 1)

In [71]:
# Avoid symmetric alternative solutions
m.chgVarLb(y[(11,1)],1)

Objective

In [72]:
# Objective function: minimize total distance travel
m.setObjective(scip.quicksum(dist[i,j]*x[(i,j,k)] for i,j in edges for k in dayType))

In [73]:
m.optimize()

In [74]:
m.getStatus()

'infeasible'